In [29]:
import numpy
import imaginet.data_provider as dp
import imaginet.driver
from funktional.util import grouper
from imaginet.tokens import tokenize
from imaginet.models import predictor_v
from scipy.spatial.distance import cosine

In [6]:
prov = dp.getDataProvider('coco', root='/home/gchrupala/repos/reimaginet')
sents = list(prov.iterSentences(split='val')) + list(prov.iterSentences(split='restval'))

In [8]:
modeldir = "/home/gchrupala/repos/reimaginet/run-104-repro/"
M = imaginet.driver.load(modeldir, model_name='model.6.pkl.gz')

In [33]:
import json
rows = [ json.loads(line) for line in open("crowd/job_886896_meta.json")]

In [13]:
batcher = M['batcher']
mapper = batcher.mapper
scaler = M['scaler']
model = M['model']
predict_v = predictor_v(model)
batch_size=64

In [16]:
inputs = list(mapper.transform([tokenize(sent['data']['desc']) for sent in rows ]))
preds = numpy.vstack([ predict_v(batcher.batch_inp(batch))
               for batch in grouper(inputs, batch_size) ])

In [86]:
img_fs = {}
for img in prov.iterImages(split='val'):
    img_fs[img['cocoid']] = scaler.transform([ img['feat'] ])[0]
for img in prov.iterImages(split='test'):
    img_fs[img['cocoid']] = scaler.transform([ img['feat'] ])[0]
for img in prov.iterImages(split='restval'):
    img_fs[img['cocoid']] = scaler.transform([ img['feat'] ])[0]

In [112]:
def features(cocoid):
    if cocoid in img_fs:
        return img_fs[cocoid]
    else:
        raise KeyError(cocoid)

def evaluate(M, rows):
    batcher = M['batcher']
    mapper = batcher.mapper
    scaler = M['scaler']
    model = M['model']
    predict_v = predictor_v(model)
    batch_size=64
    def response(row):
        inputs = list(mapper.transform([tokenize(row['data']['desc']) ]))
        pred = predict_v(batcher.batch_inp(inputs))[0]
        return 1+numpy.argmin([ cosine(pred, features(cocoid)) for cocoid in row['meta']['candidates']])
    preds = numpy.array([ response(row) for row in rows ])
    target = numpy.array([ row['meta']['response'] for row in rows])
    return numpy.mean(preds==target)

In [113]:
evaluate(M, rows)

0.83577712609970678

In [115]:
for i in range(1,7):
    modeldir = "/home/gchrupala/repos/reimaginet/run-104-repro/"
    M = imaginet.driver.load(modeldir, model_name='model.{}.pkl.gz'.format(i))
    print i, evaluate(M, rows)

1 0.800586510264
2 0.809384164223
3 0.821114369501
4 0.82991202346
5 0.838709677419
6 0.8357771261
